In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
DS_FLUX_WGT.sel(igg=1).sel(g=1).to_dataframe();

In [14]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')
    
    html_download = f'Download: <a href="./{fname}">{fname}</a>'
    
    display.display(display.HTML(html_download))
    
    
    
    


In [15]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    s_outputs_byg = 'Outputs by g-interval'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_outputs_byg))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
    
    # Outputs by g-interval
    show_html(climahtml.getHTML_idanchor(s_outputs_byg))
    show_markdown(climahtml.getMarkdown_sectitle(s_outputs_byg))
    script_fluxcoolr_excelfile()
   
    

In [16]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 913ed72
molecule = {'co2': 0.0004}
band = [4]
atmpro = trp
tsfc = 300


# LBLNEW case compaired against

...

# Cooling Rate by Layer

CRD            CLIRAD `coolr_bands`           
            layer     coolrg                layer     coolrg
pressure                                                    
0.000312        1  76.911573                    1  42.494435
0.000750        2  41.020532                    2  36.550949
0.001052        3  33.259869                    3  33.971339
0.001476        4  26.397425                    4  30.396372
0.002070        5  20.664232                    5  25.869736
0.002904        6  16.081310                    6  20.634544
0.004074        7  12.530285                    7  15.281001
0.005714        8   9.827018                    8  10.545013
0.008015        9   7.770893                    9   7.081211
0.011243       10   6.180803                   10   5.086612
0.015771       11   4.903649                   11   4.126531
0.022122       12   3.810097                   12   3.577861
0.031031       13   2.745222                   13   2.930079
0.043528       14   1.399978                   14   1.806463
0.061057       15   0.979141                   15   1.385372
0.085645       16   1.752999                   16   1.982340
0.120136       17   2.555489                   17   2.549577
0.168516       18   3.176324                   18   2.903992
0.236378       19   3.831257                   19   3.417505
0.331549       20   4.628079                   20   4.366902
0.465100       21   5.780301                   21   5.896071
0.652400       22   7.472781                   22   7.865442
0.915100       23   7.603271                   23   7.936369
1.283650       24   6.451719                   24   6.349627
1.800600       25   5.474882                   25   5.114304
2.525700       26   4.427503                   26   4.335204
3.542800       27   3.636540                   27   3.969940
4.969550       28   3.059310                   28   3.387805
6.970850       29   2.591584                   29   2.666767
9.778100       30   2.179984                   30   2.191509
13.715850      31   1.799270                   31   2.061755
19.239350      32   1.443215                   32   1.793160
26.987250      33   1.121665                   33   1.292743
37.855300      34   0.874324                   34   0.890750
53.100050      35   0.507275                   35   0.581503
73.887500      36   0.034923                   36   0.132711
97.662500      37  -0.289156                   37  -0.288601
121.437500     38  -0.219236                   38  -0.262461
145.212500     39   0.020826                   39  -0.003700
168.987500     40   0.075532                   40   0.053215
192.762500     41   0.099899                   41   0.070273
216.537500     42   0.111780                   42   0.074473
240.312500     43   0.116942                   43   0.077518
264.087500     44   0.118045                   44   0.081547
287.862500     45   0.116553                   45   0.086172
311.637500     46   0.113290                   46   0.090517
335.412500     47   0.108849                   47   0.092883
359.187500     48   0.103653                   48   0.094445
382.962500     49   0.097977                   49   0.092952
406.737500     50   0.092141                   50   0.090056
430.512500     51   0.086304                   51   0.085897
454.287500     52   0.080623                   52   0.079104
478.062500     53   0.075138                   53   0.073323
501.837500     54   0.069950                   54   0.065076
525.612500     55   0.065174                   55   0.059512
549.387500     56   0.060815                   56   0.052048
573.162500     57   0.057038                   57   0.046953
596.937500     58   0.054224                   58   0.042212
620.712500     59   0.052987                   59   0.039695
644.487500     60   0.054949                   60   0.040790
668.262500     61   0.061355                   61   0.046740
692.037500     62   0.069008                   62   0.054589
715.812500     63   0.067517                   63

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -16.254068   0.000000 -16.254068 -15.665399  9.676178e-07   
0.000624    2     -16.254436   0.006056 -16.248379 -15.665795  3.540051e-03   
0.000876    3     -16.254603   0.007445 -16.247157 -15.665956  4.790432e-03   
0.001229    4     -16.254812   0.009044 -16.245768 -15.666166  6.419081e-03   
0.001723    5     -16.255074   0.010854 -16.244221 -15.666437  8.472151e-03   
0.002417    6     -16.255405   0.012883 -16.242522 -15.666789  1.095095e-02   
0.003391    7     -16.255824   0.015157 -16.240667 -15.667247  1.378956e-02   
0.004757    8     -16.256362   0.017721 -16.238640 -15.667848  1.686195e-02   
0.006672    9     -16.257057   0.020647 -16.236411 -15.668638  2.004566e-02   
0.009359    10    -16.257970   0.024033 -16.233937 -15.669691  2.335227e-02   
0.013128    11    -16.259189   0.028011 -16.231178 -15.671108  2.704089e-02   
0.018415    12    -16.260851   0.032745 -16.228107 -15.673059  3.157607e-02   
0.025830    13    -16.263185   0.038425 -16.224760 -15.675796  3.745609e-02   
0.036232    14    -16.266652   0.045275 -16.221377 -15.679809  4.508040e-02   
0.050823    15    -16.272641   0.053684 -16.218958 -15.686472  5.486600e-02   
0.071291    16    -16.282331   0.065747 -16.216584 -15.696891  6.864377e-02   
0.100000    17    -16.295514   0.084892 -16.210622 -15.710848  8.934336e-02   
0.140271    18    -16.312196   0.113765 -16.198431 -15.728311  1.189695e-01   
0.196760    19    -16.332902   0.155726 -16.177177 -15.749704  1.597967e-01   
0.275997    20    -16.357731   0.216515 -16.141216 -15.774957  2.171305e-01   
0.387100    21    -16.385269   0.304963 -16.080306 -15.802364  3.020157e-01   
0.543100    22    -16.408114   0.434624 -15.973490 -15.824148  4.327667e-01   
0.761700    23    -16.399970   0.619986 -15.779984 -15.815095  6.274084e-01   
1.068500    24    -16.348487   0.844825 -15.503661 -15.764347  8.651185e-01   
1.498800    25    -16.263914   1.089110 -15.174804 -15.681577  1.106036e+00   
2.102400    26    -16.142986   1.359641 -14.783346 -15.562317  1.352491e+00   
2.949000    27    -15.993169   1.653839 -14.339330 -15.413232  1.638209e+00   
4.136600    28    -15.814574   1.986831 -13.827743 -15.232696  2.016221e+00   
5.802500    29    -15.601602   2.377576 -13.224026 -15.015101  2.467237e+00   
8.139200    30    -15.348183   2.841504 -12.506679 -14.749582  2.939952e+00   
11.417000   31    -15.048074   3.387837 -11.660237 -14.428143  3.469518e+00   
16.014700   32    -14.695350   4.015049 -10.680300 -14.043236  4.207622e+00   
22.464000   33    -14.284245   4.706511  -9.577734 -13.590374  5.124814e+00   
31.510500   34    -13.800127   5.424394  -8.375733 -13.045793  5.965708e+00   
44.200100   35    -13.154004   6.092532  -7.061472 -12.298919  6.557925e+00   
62.000000   36    -12.436785   6.444915  -5.991870 -11.441556  6.926802e+00   
85.775000   37    -12.199267   6.305751  -5.893515 -11.117703  6.976746e+00   
109.550000  38    -12.862807   6.154936  -6.707871 -11.831024  6.877191e+00   
133.325000  39    -14.076385   6.751076  -7.325309 -13.164146  7.471062e+00   
157.100000  40    -15.303670   8.037015  -7.266656 -14.492342  8.788836e+00   
180.875000  41    -16.528801   9.474868  -7.053934 -15.801877  1.024826e+01   
204.650000  42    -17.745588  10.973001  -6.772587 -17.088005  1.173231e+01   
228.425000  43    -18.950082  12.492304  -6.457778 -18.353425  1.320750e+01   
252.200000  44    -20.139331  14.010899  -6.128432 -19.594851  1.466726e+01   
275.975000  45    -21.311691  15.515712  -5.795979 -20.813269  1.611536e+01   
299.750000  46    -22.466145  16.998415  -5.467729 -22.009388  1.755420e+01   
323.525000  47    -23.602153  18.453483  -5.148670 -23.180678  1.898044e+01   
347.300000  48    -24.719490  19.877374  -4.842116 -24.331197  2.039257e+01

# Outputs by g-interval

In [17]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')